<a href="https://colab.research.google.com/github/plthiyagu/AI-Engineering/blob/master/12-Vision%20and%20Image/Geospatial_Analysis_Folium.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://www.analyticsvidhya.com/blog/2021/05/geospatial-analysis-getting-started-with-folium-in-python/?utm_source=feedburner&utm_medium=email&utm_campaign=Feed%3A+AnalyticsVidhya+%28Analytics+Vidhya%29


https://georgetsilva.github.io/posts/mapping-points-with-folium/

In [ ]:
!pip install folium

In [21]:
import numpy as np
import pandas as pd
import folium
import requests
from xml.etree import ElementTree

In [7]:
# Create a map
chennai = folium.Map(location=[13.067439, 80.237617], tiles='openstreetmap', zoom_start=12)
chennai

In [8]:
# Create a map
kol = folium.Map(location=[22.57, 88.36], tiles='openstreetmap', zoom_start=12)
kol

In [9]:
#add marker for a place

#victoria memorial
tooltip_1 = "This is Victoria Memorial"
tooltip_2 ="This is Eden Gardens"

folium.Marker(
    [22.54472, 88.34273], popup="Victoria Memorial", tooltip=tooltip_1).add_to(kol)

folium.Marker(
    [22.56487826917627, 88.34336378854425], popup="Eden Gardens", tooltip=tooltip_2).add_to(kol)

kol

In [10]:
folium.Marker(
    location=[22.55790780507432, 88.35087264462007],
    popup="Indian Museum",
    icon=folium.Icon(color="red", icon="info-sign"),
).add_to(kol)

kol

In [11]:
kol2 = folium.Map(location=[22.55790780507432, 88.35087264462007], tiles="Stamen Toner", zoom_start=13)
kol2

In [12]:
#adding circle

folium.Circle(
    location=[22.585728381244373, 88.41462932675563],
    radius=1500,
    popup="Salt Lake",
    color="blue",
    fill=True,
).add_to(kol2)

folium.Circle(
    location=[22.56602918189088, 88.36508424354102],
    radius=2000,
    popup="Old Kolkata",
    color="red",
    fill=True,
).add_to(kol2)


kol2

In [13]:
# Create a map
india = folium.Map(location=[20.180862078886562, 78.77642751195584], tiles='openstreetmap', zoom_start=5)
india

In [14]:
#adding 3 locations, Mumbai, Delhi and Kolkata
loc= [(19.035698150834815, 72.84981409864244),(28.61271068361265, 77.22359851696532) ,
      (22.564213404457185, 88.35872006950966)]

In [15]:
folium.PolyLine(locations = loc,
                line_opacity = 0.5).add_to(india)

india

In [16]:
df_state=pd.read_csv('https://raw.githubusercontent.com/plthiyagu/Personnel/master/Dataset/Indian%20Census%20Data%20with%20Geospatial%20indexing/state%20wise%20centroids_2011.csv')


In [17]:
df_state.head()

,State,Longitude,Latitude
0,Andaman and Nicobar,92.889579,11.942373
1,Andhra Pradesh,79.916203,16.554124
2,Arunachal Pradesh,94.545327,27.725765
3,Assam,92.657310,26.321341
4,Bihar,85.636774,25.771394


In [18]:
#creating a new map for India, for all states population centres to be plotted
# Create a map
india2 = folium.Map(location=[20.180862078886562, 78.77642751195584], tiles='openstreetmap', zoom_start=4.5)

In [19]:
#adding the markers

for i in range (0,35):
    state=df_state["State"][i]
    lat=df_state["Latitude"][i]
    long=df_state["Longitude"][i]
    folium.Marker(
    [lat, long], popup=state, tooltip=state).add_to(india2)

In [20]:
india2

In [22]:
GetAllCountersUrl = "http://webservices.commuterpage.com/counters.cfc?wsdl&method=GetAllCounters"

In [23]:
xmlfile = open('xml_getallcounters.xml', 'w')
xmldata = requests.get(GetAllCountersUrl)
xmlfile.write(xmldata.text)
xmlfile.close()

In [24]:
xml_data = 'xml_getallcounters.xml'

In [25]:
tree = ElementTree.parse(xml_data)
counter = tree.find('counter')
name = counter.find('name')
name.text

'110 Trail'

In [26]:
counter.find('latitude').text

'38.885315'

In [27]:
counter.find('longitude').text

'-77.065022'

In [28]:
id = []
name = []
latitude = []
longitude = []
region = []


for c in tree.findall('counter'):
    id.append(c.attrib['id'])
    name.append(c.find('name').text)
    latitude.append(c.find('latitude').text)
    longitude.append(c.find('longitude').text)
    region.append(c.find('region/name').text)

df_counters = pd.DataFrame(
    {'ID' : id,
     'Name' : name,
     'latitude' : latitude,
     'longitude' : longitude,
     'region' : region
    })
df_counters.head()

,ID,Name,latitude,longitude,region
0,33,110 Trail,38.885315,-77.065022,Arlington
1,30,14th Street Bridge,38.874260,-77.044610,Arlington
2,43,15th Street NW,38.907470,-77.034610,DC
3,24,Ballston Connector,38.882950,-77.121235,Arlington
4,59,Bluemont Connector,38.880440,-77.119290,Arlington


In [32]:
locations = df_counters[['latitude', 'longitude']]
locationlist = locations.values.tolist()
len(locationlist)
locationlist[7]

['38.943070', '-77.115660']

In [33]:
import sys
print(sys.getrecursionlimit())

1000


In [34]:
sys.setrecursionlimit(1500)

In [ ]:
map = folium.Map(location=[38.9, -77.05], zoom_start=12)
for point in range(0, len(locationlist)):
  folium.Marker(locationlist[point], popup=df_counters['Name'][point]).add_to(map)
map

In [ ]:
map2 = folium.Map(location=[38.9, -77.05], tiles='CartoDB dark_matter', zoom_start=11)

marker_cluster = folium.MarkerCluster().add_to(map2)

for point in range(0, len(locationlist)):
    folium.Marker(locationlist[point], popup=df_counters['Name'][point]).add_to(marker_cluster)
map2

In [38]:
def regioncolors(counter):
    if counter['region'] == 'Arlington':
        return 'green'
    elif counter['region'] == 'Alexandria':
        return 'blue'
    elif counter['region'] == 'DC':
        return 'red'
    else:
        return 'darkblue'
df_counters["color"] = df_counters.apply(regioncolors, axis=1)
df_counters.head()


,ID,Name,latitude,longitude,region,color
0,33,110 Trail,38.885315,-77.065022,Arlington,green
1,30,14th Street Bridge,38.874260,-77.044610,Arlington,green
2,43,15th Street NW,38.907470,-77.034610,DC,red
3,24,Ballston Connector,38.882950,-77.121235,Arlington,green
4,59,Bluemont Connector,38.880440,-77.119290,Arlington,green


In [39]:
map3 = folium.Map(location=[38.9, -77.05], tiles='CartoDB positron', zoom_start=11)

marker_cluster = folium.MarkerCluster().add_to(map3)

for point in range(0, len(locationlist)):
    folium.Marker(locationlist[point], popup='ID:'+df_counters['ID'][point]+' '+df_counters['Name'][point], icon=folium.Icon(color=df_counters["color"][point], icon_color='white', icon='male', angle=0, prefix='fa')).add_to(marker_cluster)
map3

AttributeError: ignored